In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as math
import os

exec(open("../header.py").read())

Header initialized


In [2]:
raw_train_data = pd.read_csv(processed_root("03-bag-of-words/bow_train_data.csv"))
raw_val_data = pd.read_csv(processed_root("03-bag-of-words/bow_val_data.csv"))
raw_test_data = pd.read_csv(processed_root("03-bag-of-words/bow_test_data.csv"))

In [5]:
raw_train_data['poetry_text'].values[0]

'(excerpt)\n"Throughout these infinite orbs of mingling light,\nOf which yon earth is one, is wide diffus\'d\nA Spirit of activity and life,\nThat knows no term, cessation, or decay;\nThat fades not when the lamp of earthly life,\nExtinguish\'d in the dampness of the grave,\nAwhile there slumbers, more than when the babe\nIn the dim newness of its being feels\nThe impulses of sublunary things,\nAnd all is wonder to unpractis\'d sense:\nBut, active, steadfast and eternal, still\nGuides the fierce whirlwind, in the tempest roars,\nCheers in the day, breathes in the balmy groves,\nStrengthens in health, and poisons in disease;\nAnd in the storm of change, that ceaselessly\nRolls round the eternal universe and shakes\nIts undecaying battlement, presides,\nApportioning with irresistible law\nThe place each spring of its machine shall fill;\nSo that when waves on waves tumultuous heap\nConfusion to the clouds, and fiercely driven\nHeaven\'s lightnings scorch the uprooted ocean-fords,\nWhilst

### Feature Engineering

Average word length, average words per line, average syllables per line, rhyme (different rhyme schemes, or number of rhymes etc.), genre (nature, love, etc., use embedding space to find out how close the poems are to each genre) 

In [56]:
import string
import nltk
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.stem.snowball import SnowballStemmer
import enchant
import re

In [57]:
# intializing checkeres
d = enchant.Dict("en_US")
lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer("english")

In [58]:
def word_suggestions(w):
    if d.check(w) == False:
        w = d.suggest(w)[0]
    return w

In [95]:
def str_cleanup(s):
    """given a line of a poem, get rid of punctuation, all lowercase, etc"""
    # all lower case
    s = s.lower()
    # first split into words
    s = s.split(" ")
    if '' in s:
        return 
    # get rid of punctuation
    s = list(map(lambda x: re.sub("[.,-/\"'();:{}=!?%^&*]","",x), s))
    # spell checks
    s = list(map(lambda x: word_suggestions(x), s))
    return s

In [96]:
first_poem = raw_train_data['poetry_text'].values[0]
first_poem = first_poem.split("\n")

#### Average word length

In [93]:
def ave_word_len(poem):
    lines = poem.split("\n")
    p = list(map(str_cleanup, lines))
    # get rid of none types
    p_none = filter(None.__ne__, p)
    p = list(p_none)
    word_lens = []
    for line in p:
        word_lens.append()
    return np.mean(list(map(len, p)))

In [94]:
ave_word_len(raw_train_data['poetry_text'].values[0])

6.904255319148936

#### Average words per line

In [97]:
def ave_line_len(poem):
    lines = poem.split("\n")
    p = list(map(str_cleanup, lines))
    # get rid of none types
    p_none = filter(None.__ne__, p)
    p = list(p_none)
    return np.mean(list(map(len, p)))

In [98]:
ave_line_len(raw_train_data['poetry_text'].values[0])

6.904255319148936